In [1]:
import cv2
import numpy as np
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [2]:
# --- Dirsek açısı (2D, 0-180°) ---
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ang = np.degrees(np.arctan2(c[1]-b[1], c[0]-b[0]) -
                     np.arctan2(a[1]-b[1], a[0]-b[0]))
    ang = abs(ang)
    if ang > 180:
        ang = 360 - ang
    return float(round(ang, 1))

# --- Alfa kanallı PNG bindirme (logo için) ---
def overlay_rgba(bg, fg, x, y):
    if fg is None: 
        return
    fh, fw = fg.shape[:2]
    x2, y2 = x + fw, y + fh
    if x < 0 or y < 0 or x2 > bg.shape[1] or y2 > bg.shape[0]:
        return
    if fg.shape[2] == 4:
        alpha = fg[:, :, 3] / 255.0
        for c in range(3):
            bg[y:y2, x:x2, c] = (alpha * fg[:, :, c] + (1 - alpha) * bg[y:y2, x:x2, c]).astype(bg.dtype)
    else:
        bg[y:y2, x:x2] = fg

# --- Dinamik HUD: REPS (solda) + STAGE (sağda) + marka satırı ---
def draw_hud(image, counter, stage_text, brand='github.com/yarencelikk', logo=None):
    h, w = image.shape[:2]
    pad = 10
    # Yazı stilleri
    f_lbl, s_lbl, t_lbl = cv2.FONT_HERSHEY_SIMPLEX, 0.65, 1
    f_val, s_val, t_val = cv2.FONT_HERSHEY_SIMPLEX, 2.0, 2

    reps = str(counter)
    stage_text = (stage_text or '-').upper()

    # Metin ölçüleri
    (rep_w, rep_h), _   = cv2.getTextSize(reps,       f_val, s_val, t_val)
    (stag_w, stag_h), _ = cv2.getTextSize(stage_text, f_val, s_val, t_val)
    (rl_w, rl_h), _     = cv2.getTextSize('REPS',     f_lbl, s_lbl, t_lbl)
    (sl_w, sl_h), _     = cv2.getTextSize('STAGE',    f_lbl, s_lbl, t_lbl)

    left_w  = pad*2 + max(rep_w,  rl_w) + 10
    right_w = pad*2 + max(stag_w, sl_w) + 10
    hud_w   = min(w, left_w + right_w + 20)
    hud_h   = pad*2 + max(rep_h, stag_h) + 24

    # Kutu
    cv2.rectangle(image, (0, 0), (hud_w, hud_h), (245,117,16), -1)

    # Sol: REPS
    cv2.putText(image, 'REPS', (pad, pad + 18), f_lbl, s_lbl, (0,0,0), t_lbl, cv2.LINE_AA)
    cv2.putText(image, reps,   (pad, hud_h - pad), f_val, s_val, (255,255,255), t_val, cv2.LINE_AA)

    # Ayırıcı
    x_sep = left_w + 10
    cv2.line(image, (x_sep, 0), (x_sep, hud_h), (200,200,200), 1)

    # Sağ: STAGE (sağa hizalı)
    cv2.putText(image, 'STAGE', (x_sep + pad, pad + 18), f_lbl, s_lbl, (0,0,0), t_lbl, cv2.LINE_AA)
    x_stage = hud_w - pad - stag_w
    cv2.putText(image, stage_text, (x_stage, hud_h - pad), f_val, s_val, (255,255,255), t_val, cv2.LINE_AA)

    # Marka satırı
    if brand:
        (bw, bh), _ = cv2.getTextSize(brand, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.putText(image, brand, (pad, hud_h + bh + 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255,255,255), 1, cv2.LINE_AA)

    # Logo (opsiyonel)
    if logo is not None:
        target_h = 26
        scale = target_h / logo.shape[0]
        logo = cv2.resize(logo, (int(logo.shape[1]*scale), target_h), interpolation=cv2.INTER_AREA)
        overlay_rgba(image, logo, w - logo.shape[1] - 10, 10)


In [3]:
SIDE = "left"         # "left" | "right"
ANGLE_UP_THR = 30     # bükülmüş sayılacak eşik
ANGLE_DOWN_THR = 160  # açılmış sayılacak eşik

def get_triplet(landmarks, side="left"):
    if side == "right":
        S = mp_pose.PoseLandmark.RIGHT_SHOULDER.value
        E = mp_pose.PoseLandmark.RIGHT_ELBOW.value
        W = mp_pose.PoseLandmark.RIGHT_WRIST.value
    else:
        S = mp_pose.PoseLandmark.LEFT_SHOULDER.value
        E = mp_pose.PoseLandmark.LEFT_ELBOW.value
        W = mp_pose.PoseLandmark.LEFT_WRIST.value
    sh = [landmarks[S].x, landmarks[S].y]
    el = [landmarks[E].x, landmarks[E].y]
    wr = [landmarks[W].x, landmarks[W].y]
    return sh, el, wr


In [6]:
cap = cv2.VideoCapture(0)

counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Selfie görünüm (tek flip)
        frame = cv2.flip(frame, 1)

        # MediaPipe
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_rgb.flags.writeable = False
        results = pose.process(img_rgb)

        image = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)  # çizimleri BGR'de yapacağız
        h, w = image.shape[:2]

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark

            # Sol/sağ eklem üçlüsü
            shoulder, elbow, wrist = get_triplet(lm, SIDE)

            # Açı
            angle = calculate_angle(shoulder, elbow, wrist)

            # Açı etiketini dirseğe (piksel) yaz
            ex, ey = (np.array(elbow) * [w, h]).astype(int)
            cv2.putText(image, f"{angle:.1f}", (ex, ey),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)

            # Rep sayacı mantığı
            if angle > ANGLE_DOWN_THR:
                stage = "down"
            if angle < ANGLE_UP_THR and stage == "down":
                stage = "up"
                counter += 1

            # İskelet
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )

        draw_hud(image, counter, stage, brand='github.com/yarencelikk', logo=None)

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
